In [1]:
import os
os.environ["https_proxy"] = "http://192.168.1.12:7891"

In [2]:
import json
import re
from datasets import load_dataset
import sys
sys.path.append("../..")
from utils.sys_prompts import SYS_PROMPT_formatter_deepseek_concise_2 as SYSTEM_PROMPT
import random

INSTRUCTION_RESPONSE_FORMAT = """\
<instruction>
{instruction}
</instruction>
<response>
{response}
</response>
"""

def get_natural_thinking_dataset(url, split="train", output_file="dataset.json"):
    dataset = load_dataset(url, split=split)
    
    def check_structure_markdown(text):
        structure_markdown_regex = r"^#{1,6} .*$"
        res = re.findall(structure_markdown_regex, text["response1"], re.MULTILINE)
        return len(res) > 0
    
    def check_length(text):
        text_lenth = len(text["response1"])
        return (text_lenth > 50) and (text_lenth < 2000)
    
    dataset = dataset.filter(check_structure_markdown)
    dataset = dataset.filter(check_length)
    print("\033[92m Number of data after filtering: \033[0m", len(dataset))
    
    def formatting_prompts_func(examples):
        instruction = examples["context"][0]['content'].strip()
        output = examples["response1"].strip()
        
        return {
            'prompt': [
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': INSTRUCTION_RESPONSE_FORMAT.format(instruction=instruction, response=output)}
            ]
        }
    
    dataset = dataset.map(formatting_prompts_func, batched=False)
    dataset = dataset.remove_columns([col for col in dataset.column_names if col != "prompt"])
    # dataset = dataset.shuffle(seed=42).select(range(min(1000, len(dataset))))  # Ensure we get up to 1000 samples
    
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(dataset.to_list(), f, indent=4, ensure_ascii=False)
    
    return dataset


/root/workspace/development/frugal-agent/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

dataset = get_natural_thinking_dataset("nvidia/HelpSteer3", "train", "HelpSteer3-markdown.json")

Filter: 100%|██████████| 2808/2808 [00:00<00:00, 23688.63 examples/s]


 Number of data after filtering:  1077


Map: 100%|██████████| 1077/1077 [00:00<00:00, 11246.49 examples/s]


In [4]:
dataset.to_list()[-1]

{'prompt': [{'content': 'You are a meticulous organizational assistant specialized in structuring instruction-response pairs into a standardized markdown format. Please carefully process the input according to the following specifications:\n\n---\ntags:\n  - {general_tag} {general_tag}/{sub_tag}\n---\n# Instruction\n[The original instruction text]\n\n# Summary\n[A brief yet comprehensive summary of the response]\n\n## Details\n[The original response content]\n\nHere are Processing Guidelines:\n- The `tags` section consists of pairs of general tags and sub tags in the following format:  `{general_tag} {general_tag}/{sub_tag}`. For example: `environment environment/renewable_energy`.\n- Keep the heaidng levels in the original response and adjust heading levels as needed to maintain proper hierarchy and avoid jumping heading levels.\n\nThe instructions and responses are enclosed within `<instruction>` and `<response>` XML tags, respectively. Please process the following instruction-respon

In [8]:
for i in range(0):
    print('-'*100)
    print(dataset[i]['prompt'][1]['content'])

In [14]:
import json

file = "HelpSteer3.json"

with open(file, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"Number of entries: {len(data)}")



Number of entries: 2808
